In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

from custom_estimator import Estimator
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import os
import glob
from sklearn.externals import joblib
from encoding import FreqeuncyEncoding
import warnings
warnings.filterwarnings("ignore")
from sklearn.externals import joblib

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train=pd.read_pickle('../train_v2.pkl')

In [3]:
from sklearn.model_selection import GroupKFold
folds=GroupKFold(n_splits=5)
folds1=[(x,y) for x,y in  folds.split(train,train.redemption_status,train.coupon_id)]
folds2=[(x,y) for x,y in  folds.split(train,train.redemption_status,train.campaign_id)]

In [4]:
from sklearn.metrics import roc_auc_score

In [5]:
target=pd.read_csv('meta_target.csv')

In [6]:
oof1=pd.read_csv('OOF_baseline_lgb_grp_k_coup_more_feats.csv')
oof2=pd.read_csv('OOF_baseline_lgb_grp_k_coup_more_feats_v2.csv')
oof3=pd.read_csv('OOF_baseline_xgb_grp_k_coup_more_feats.csv')
oof4=pd.read_csv('OOF_baseline_xgb_grp_k_coup_more_feats_v2.csv')
oof5=pd.read_csv('OOF_baseline_lgb_grp_k_camp_more_feats.csv')
oof6=pd.read_csv('OOF_baseline_lgb_grp_k_camp_more_feats_v2.csv')
oof7=pd.read_csv('OOF_baseline_xgb_grp_k_camp_more_feats.csv')
oof8=pd.read_csv('OOF_baseline_xgb_grp_k_camp_more_feats_v2.csv')

oof9=pd.read_csv('OOF_baseline_catb_grp_k_coup_more_feats.csv')
oof10=pd.read_csv('OOF_baseline_catb_grp_k_coup_more_feats_v2.csv')
oof11=pd.read_csv('OOF_baseline_catb_grp_k_camp_more_feats.csv')
oof12=pd.read_csv('OOF_baseline_catb_grp_k_camp_more_feats_v2.csv')


In [7]:
pred1=pd.read_csv('baseline_lgb_grp_k_coup_more_feats.csv')
pred2=pd.read_csv('baseline_lgb_grp_k_coup_more_feats_v2.csv')
pred3=pd.read_csv('baseline_xgb_grp_k_coup_more_feats.csv')
pred4=pd.read_csv('baseline_xgb_grp_k_coup_more_feats_v2.csv')
pred5=pd.read_csv('baseline_lgb_grp_k_camp_more_feats.csv')
pred6=pd.read_csv('baseline_lgb_grp_k_camp_more_feats_v2.csv')
pred7=pd.read_csv('baseline_xgb_grp_k_camp_more_feats.csv')
pred8=pd.read_csv('baseline_xgb_grp_k_camp_more_feats_v2.csv')

pred9=pd.read_csv('baseline_catb_grp_k_coup_more_feats.csv')
pred10=pd.read_csv('baseline_catb_grp_k_coup_more_feats_v2.csv')
pred11=pd.read_csv('baseline_catb_grp_k_camp_more_feats.csv')
pred12=pd.read_csv('baseline_catb_grp_k_camp_more_feats_v2.csv')


In [8]:
meta_train_coup=pd.DataFrame({
    'coup_lgb_v1':oof1.redemption_status,
    'coup_lgb_v2':oof2.redemption_status,
    'coup_xgb_v1':oof3.redemption_status,
    'coup_xgb_v2':oof4.redemption_status,
    'coup_catb_v1':oof9.redemption_status,
    'coup_catb_v2':oof10.redemption_status,    
})
meta_train_camp=pd.DataFrame({
'camp_lgb_v1':oof5.redemption_status,
    'camp_lgb_v2':oof6.redemption_status,
    'camp_xgb_v1':oof7.redemption_status,
    'camp_xgb_v2':oof8.redemption_status,
    'camp_catb_v1':oof11.redemption_status,
    'camp_catb_v2':oof12.redemption_status,    

})


In [9]:
meta_test_coup=pd.DataFrame({
    'coup_lgb_v1':pred1.redemption_status,
    'coup_lgb_v2':pred2.redemption_status,
    'coup_xgb_v1':pred3.redemption_status,
    'coup_xgb_v2':pred4.redemption_status,
    'coup_catb_v1':pred9.redemption_status,
    'coup_catb_v2':pred10.redemption_status,    

})
meta_test_camp=pd.DataFrame({
'camp_lgb_v1':pred5.redemption_status,
    'camp_lgb_v2':pred6.redemption_status,
    'camp_xgb_v1':pred7.redemption_status,
    'camp_xgb_v2':pred8.redemption_status,    
    'camp_catb_v1':pred11.redemption_status,
    'camp_catb_v2':pred12.redemption_status,    

})


In [10]:
roc_auc_score(target['redemption_status'],pd.concat((meta_train_camp,meta_train_coup),axis=1).mean(axis=1))

0.9604095155510043

In [11]:
sub=pd.read_csv('../sample_submission_Byiv0dS.csv')
preds=pd.concat((meta_test_camp,meta_test_coup),axis=1).mean(axis=1)
pd.DataFrame({'id':pred1['id'],'redemption_status': preds}).to_csv('final_sub_2.csv',index=False)